# Projeto 1 - Ciência dos Dados

Nome: Layne Silva

Nome: Lidia Domingos

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [253]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import nltk 
from nltk.stem import RSLPStemmer
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\lidia\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [254]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
C:\Users\lidia\Desktop\Ciência dos Dados\Projeto1\PROJETO-1-CDADOS


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [255]:
filename = 'XIAOMI.xlsx'

In [256]:
train = pd.read_excel(filename)
train.head(5)

,TREINAMENTO,CLASSIFICAÇÃO
0,quem aqui tb pronuncia xiaomi xaiomi,0
1,xiaomi mi true airdots 2s earphone(aliexpress)...,0
2,@felpsflip carreguei meu xiaomi uma vez na vid...,1
3,"#promo #\n$133.99 ~ 111,85 € ✂️cupão/cupom/cup...",0
4,"#promo #\n$299.99 ~ 250,41 € ✂️cupão/cupom/cup...",0


In [257]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)

,Teste,CLASSIFICAÇÃO
0,"#promo #\n$11.99 ~ 10,01 € ✂️cupão/cupom/cupon...",0
1,"💠xiaomi poco m3 \n\n💵 por: r$ 728,27 64gb\n💵 ...",0
2,"@_faccin_ ""quem ai tem um xiaomi pra ligar o p...",0
3,comprei mais capinha pro xiaomi https://t.co/7...,0
4,#promoção #oferta \n\n💵 por: r$ 1420 \n🛒 link:...,0


In [258]:
test.iloc[:,0:1]

,Teste
0,"#promo #\n$11.99 ~ 10,01 € ✂️cupão/cupom/cupon..."
1,"💠xiaomi poco m3 \n\n💵 por: r$ 728,27 64gb\n💵 ..."
2,"@_faccin_ ""quem ai tem um xiaomi pra ligar o p..."
3,comprei mais capinha pro xiaomi https://t.co/7...
4,#promoção #oferta \n\n💵 por: r$ 1420 \n🛒 link:...
...,...
254,"rt @victorxama: salve rapa, tô vendendo meu xi..."
255,@mslaryhill tenho um haylou gt1 pro q é da mes...
256,@gabiiii__araujo 😱😱😱\n\n*baixou mais!!*\n\n*xi...
257,@inutializador @viniport0 @2kjuramento @pessoa...


___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

*Escolhida a empresa XIAOMI, classificamos como tweets relevantes toda e qualquer citação (seja ela positiva ou negativa) dos internautas falando sobre o produto. Tweets considerados como irrelevantes seriam tweets não relacionados diretamente ao produto.*

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [259]:
#Limpeza de mensagens removendo os caracteres: enter, :, ", ', (, ), etc. Não remover emojis.
#Correção de espaços entre palavras e/ou emojis.
#Proposta de outras limpezas/transformações que não afetem a qualidade da informação.

import re 
from nltk.tokenize.casual import TweetTokenizer #separa os emojis como palavras.

def cleanup(text):
    #import string
    punctuation = '[!-@.~[\]:"''?;\),("/]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    text_subbed = re.sub(r'http\S+', '',text_subbed)
    t = TweetTokenizer()
    text_subbed = t.tokenize(text_subbed) 
    return text_subbed

def RemoveStopWords(list_of_words):
    stopwords = nltk.corpus.stopwords.words('portuguese')
    lista_filtrada = []
    for word in list_of_words:
        if word not in stopwords:
            lista_filtrada.append(word)
    return lista_filtrada

def Suavizacao_Laplace(variavel1, variavel2, variavel3):
    y = (1 + variavel1)/(variavel2 + len(variavel3))  
    return y

def MultiplyList(List) : 
    result = 1
    for x in List: 
         result = result * x  
    return result








In [260]:
#JUNTANDO TODAS AS PALAVRAS DOS TWEETS RELEVANTES DO TREINAMENTO
train_relevante = train.loc[train["CLASSIFICAÇÃO"] == 1, :]
train_relevante_filtrada = ' '.join(train_relevante.TREINAMENTO) 

#JUNTANDO TODAS AS PALAVRAS DOS TWEETS IRRELEVANTES DO TREINAMENTO
train_irrelevante = train.loc[train["CLASSIFICAÇÃO"] == 0, :]
train_irrelevante_filtrada = ' '.join(train_irrelevante.TREINAMENTO) 


In [261]:
#LIMPANDO VARIÁVEIS CRIADAS, COLOCANDO EM MINÚSCULO E AS GUARDANDO
relevante = cleanup(train_relevante_filtrada.lower())
palavras_relevantes = RemoveStopWords(relevante)

irrelevante = cleanup(train_irrelevante_filtrada.lower())
palavras_irrelevantes = RemoveStopWords(irrelevante)

#JUNTANDO OS CONJUNTOS
palavras_total = palavras_relevantes + palavras_irrelevantes


In [262]:
#TRANSFORMANDO EM PD.SERIES
series_total = pd.Series(palavras_total)
series_train_relevante = pd.Series(palavras_relevantes)
series_train_irrelevante = pd.Series(palavras_irrelevantes)


In [263]:
#FREQUÊNCIAS RELATIVAS
palavras_total_relativas = series_total.value_counts(True)
palavras_relevante_relativas = series_train_relevante.value_counts(True)
palavras_irrelevante_relativas = series_train_irrelevante.value_counts(True)


In [264]:
#FREQUÊNCIAS ABSOLUTAS
palavras_relevante_absolutas = series_train_relevante.value_counts()
palavras_irrelevante_absolutas = series_train_irrelevante.value_counts()
palavras_total_absolutas = series_total.value_counts()


**EVENTOS**

 * $R$: comentários relevantes
 * $R^c$: comentários irrelevantes
 * $C$: comentário recebido

**PROBABILIDADES**

 * $P(R)$ : probabilidade do comentário ser relevante ;
 * $P(R^c)$: probabilidade do comentário não ser relevante, ou seja, irrelevante ;
 * $P(C)$ : probabilidade de cada palavra ocorrer na língua portuguesa ;
 * $P(C|R)$ : probabilidade do comentário existir dado o conjunto de relevantes ;
 * $P(C|R^c)$: probabilidade do comentário existir dado o conjunto de irrelevantes ; 
 * $P(R|C)$: probabilidade do comentário ser relevante dado a frase ;
 * $P(R^c|C)$: probabilidade do comentário ser irrelevante dado a frase;


**CLASSIFICADOR**



In [265]:
def funcao_classificadora(test):
    
    lista_resultados = []
    
    test['Classificação_NB'] = 'NaN'
    
    for tweet in list(test.Teste):
        
        tweet_limpo = RemoveStopWords(cleanup(tweet.lower()))
        lista_de_probabilidades1 = []
        lista_de_probabilidades2 = []
        
        for palavra in tweet_limpo:
            
            if (palavra in palavras_relevante_absolutas) & (palavra not in palavras_irrelevante_absolutas):
                Laplace = Suavizacao_Laplace(palavras_relevante_absolutas[palavra],palavras_relevante_absolutas.sum(),palavras_total_absolutas)
                lista_de_probabilidades1.append(Laplace)
                Laplace = Suavizacao_Laplace(0,palavras_irrelevante_absolutas.sum(),palavras_total_absolutas)
                lista_de_probabilidades2.append(Laplace)
                
            elif (palavra not in palavras_relevante_absolutas) & (palavra in palavras_irrelevante_absolutas):
                Laplace = Suavizacao_Laplace(palavras_irrelevante_absolutas[palavra],palavras_irrelevante_absolutas.sum(),palavras_total_absolutas)
                lista_de_probabilidades2.append(Laplace)
                Laplace = Suavizacao_Laplace(0,palavras_relevante_absolutas.sum(),palavras_total_absolutas)
                lista_de_probabilidades1.append(Laplace)
                
            elif (palavra in palavras_relevante_absolutas) & (palavra in palavras_irrelevante_absolutas):
                Laplace = Suavizacao_Laplace(palavras_relevante_absolutas[palavra],palavras_relevante_absolutas.sum(),palavras_total_absolutas)
                lista_de_probabilidades1.append(Laplace)
                Laplace = Suavizacao_Laplace(palavras_irrelevante_absolutas[palavra],palavras_irrelevante_absolutas.sum(),palavras_total_absolutas)
                lista_de_probabilidades2.append(Laplace)
            
            elif palavra == 'xiaomi':
                continue
                
            else:
                Laplace = Suavizacao_Laplace(0,palavras_relevante_absolutas.sum(),palavras_total_absolutas)
                lista_de_probabilidades1.append(Laplace)
                Laplace = Suavizacao_Laplace(0,palavras_irrelevante_absolutas.sum(),palavras_total_absolutas)
                lista_de_probabilidades2.append(Laplace)
                
            P_C_dado_R = MultiplyList(lista_de_probabilidades1)
            P_C_dado_Rc = MultiplyList(lista_de_probabilidades2)

        P_R = (palavras_relevante_absolutas.sum())/(palavras_total_absolutas.sum())
    
        P_Rc = 1 - P_R
        
        P_R_dado_C = P_R * P_C_dado_R

        P_Rc_dado_C = P_Rc * P_C_dado_Rc            
        
        if P_R_dado_C > P_Rc_dado_C:
            resultado = 1
        else:
            resultado = 0
        
        lista_resultados.append(resultado)
    test['Classificação_NB'] = lista_resultados
    return test
    

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [272]:
teste = funcao_classificadora(test)
teste

,Teste,CLASSIFICAÇÃO,Classificação_NB
0,"#promo #\n$11.99 ~ 10,01 € ✂️cupão/cupom/cupon...",Irrelevante,0
1,"💠xiaomi poco m3 \n\n💵 por: r$ 728,27 64gb\n💵 ...",Irrelevante,0
2,"@_faccin_ ""quem ai tem um xiaomi pra ligar o p...",Irrelevante,1
3,comprei mais capinha pro xiaomi https://t.co/7...,Irrelevante,0
4,#promoção #oferta \n\n💵 por: r$ 1420 \n🛒 link:...,Irrelevante,0
...,...,...,...
254,"rt @victorxama: salve rapa, tô vendendo meu xi...",Irrelevante,0
255,@mslaryhill tenho um haylou gt1 pro q é da mes...,Relevante,1
256,@gabiiii__araujo 😱😱😱\n\n*baixou mais!!*\n\n*xi...,Irrelevante,0
257,@inutializador @viniport0 @2kjuramento @pessoa...,Relevante,1


In [273]:
teste['CLASSIFICAÇÃO'] = teste['CLASSIFICAÇÃO'].astype('category')
teste['CLASSIFICAÇÃO'].cat.categories = ['Irrelevante', 'Relevante']
teste['CLASSIFICAÇÃO'].cat.categories

Index(['Irrelevante', 'Relevante'], dtype='object')

In [274]:
teste['Classificação_NB'] = teste['Classificação_NB'].astype('category')
teste['Classificação_NB'].cat.categories = ['Irrelevante', 'Relevante']
teste['Classificação_NB'].cat.categories

Index(['Irrelevante', 'Relevante'], dtype='object')

In [275]:
#comparando os resultados da classicação entre o manual e o Navy Bayes por meio do CrossTab

tabela = pd.crosstab(teste.Classificação_NB, teste.CLASSIFICAÇÃO, normalize = True).round(4)
tabela

CLASSIFICAÇÃO,Irrelevante,Relevante
Classificação_NB,,
Irrelevante,0.3089,0.0656
Relevante,0.1660,0.4595


In [276]:
verdadeiros_positivos = tabela.iloc[1,1]*100
falsos_positivos = tabela.iloc[1,0]*100
verdadeiros_negativos = tabela.iloc[0,0]*100
falsos_negativos = tabela.iloc[0,1]*100

print('A porcentagem de verdadeiros positivos é: {:.3f}%\n'.format(verdadeiros_positivos))
print('A porcentagem de falsos positivos é: {:.3f}%\n'.format(falsos_positivos))
print('A porcentagem de verdadeiros negativos é: {:.3f}%\n'.format(verdadeiros_negativos))
print('A porcentagem de falsos negativos é: {:.3f}%\n'.format(falsos_negativos))

A porcentagem de verdadeiros positivos é: 45.950%

A porcentagem de falsos positivos é: 16.600%

A porcentagem de verdadeiros negativos é: 30.890%

A porcentagem de falsos negativos é: 6.560%



___
### Concluindo

___
### Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou entre emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)


___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**